## Anthropic Function Calling

In [1]:
# !pip install langchain --upgrade -q
# !pip install defusedxml

In [2]:
# !pip install langchain-experimental

In [3]:
from langchain_experimental.llms.anthropic_functions import AnthropicFunctions
from defusedxml import ElementTree
from collections import defaultdict

In [4]:
import boto3
from langchain.chat_models import BedrockChat

bedrock_client = boto3.client('bedrock-runtime')

llm = BedrockChat(model_id="anthropic.claude-v2", client=bedrock_client)

model = AnthropicFunctions(llm=llm)

In [5]:
import json
from typing import Any

In [6]:
# Run loop and call functions
def run_loop(prompt):
    print(prompt)
    # Start function calling loop
    while True:
    # initialize variables to make bedrock api call
        bedrock = boto3.client(service_name='bedrock-runtime')
        modelId = 'anthropic.claude-v2'
        body = json.dumps({"prompt": prompt,
        "stop_sequences":["\n\nHuman:", "</function_calls>"],
        "max_tokens_to_sample": 700,
        "temperature": 0})
        accept = 'application/json'
        contentType = 'application/json'
        # bedrock api call with prompt
        partial_completion = bedrock.invoke_model(
        body=body, 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType
    )
   
        response_body = json.loads(partial_completion.get('body').read())


        partial_completion= response_body.get('completion')
        stop_reason=response_body.get('stop_reason')
        stop_seq = partial_completion.rstrip().endswith("</invoke>")
        
        # Get a completion from Claude

        # Append the completion to the end of the prommpt
        prompt += partial_completion
        if stop_reason == 'stop_sequence' and stop_seq:
            # If Claude made a function call
            print(partial_completion)
            start_index = partial_completion.find("<function_calls>")
            if start_index != -1:
                # Extract the XML Claude outputted (invoking the function)
                extracted_text = partial_completion[start_index+16:]

                # Parse the XML find the tool name and the parameters that we need to pass to the tool
                xml = ElementTree.fromstring(extracted_text)
                tool_name_element = xml.find("tool_name")
                if tool_name_element is None:
                    print("Unable to parse function call, invalid XML or missing 'tool_name' tag")
                    break
                tool_name_from_xml = tool_name_element.text.strip()
                print('tool_name_from_xml:', tool_name_from_xml)
                parameters_xml = xml.find("parameters")
                if parameters_xml is None:
                    print("Unable to parse function call, invalid XML or missing 'parameters' tag")
                    break
                param_dict = etree_to_dict(parameters_xml)
                parameters = param_dict["parameters"]

                # Call the tool we defined in tools.py
                output = call_function(tool_name_from_xml, parameters)

                # Add the stop sequence back to the prompt
                prompt += "</function_calls>"
                print("</function_calls>")

                # Add the result from calling the tool back to the prompt
                function_result = format_result(tool_name_from_xml, output)
                print(function_result)
                prompt += function_result
        else:
            # If Claude did not make a function call
            # outputted answer
            print(partial_completion)
            break

In [7]:
user_input = "Can you check a code for male patients?"
tools_string = add_tools()
prompt = create_prompt(tools_string, user_input)
run_loop(prompt)

NameError: name 'add_tools' is not defined

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
from langchain.schema import HumanMessage

In [ ]:
response = model.predict_messages(
    [HumanMessage(content="whats the weater in boston?")], functions=functions
)

In [ ]:
response